In [23]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

cuda


In [24]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
chars=sorted(set(text))
print(chars)
vocab_size =len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [25]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

e=encode('hello')
d=decode(e)
print(d)

hello


In [26]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [27]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('Inputs:')
print(x)
print('Targets:')
print(y)

Inputs:
tensor([[65, 58,  1, 65, 54, 74, 60, 61],
        [ 9,  1, 59, 68, 71,  1, 73, 61],
        [61, 58, 65, 57,  1, 55, 54, 56],
        [ 1, 76, 58,  5, 71, 58,  1, 67]], device='cuda:0')
Targets:
tensor([[58,  1, 65, 54, 74, 60, 61,  1],
        [ 1, 59, 68, 71,  1, 73, 61, 58],
        [58, 65, 57,  1, 55, 54, 56, 64],
        [76, 58,  5, 71, 58,  1, 67, 58]], device='cuda:0')


In [28]:
x= train_data[:block_size]
y=train_data[1:block_size+1]

for i in range(block_size):
    context= x[:i+1]
    target=y[i]
    print("when input is",context,"target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [29]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [30]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


[j3HeE,iPQybyYzDIcMyh0TRx.fhA 7M4,-I!ijCfzU'lCfX BUi0(Eg8Sn8hNS&-&p"EDmmRMz
 _TBYzj5S﻿;BIEa*613?eOOSOobt8﻿r_?wET﻿5aZYczAK3H﻿v8r.fntK,ixVHeAT?)aZE-q?i7Mr.lFCmKitb]Qm)YJ8
 eSq?Mu-Kc﻿gRIrH&*kY9AAFLG qJK5R?;2]z"'
"8O1"?7dvj3
xid0,JQ2Pxzk_*ur:XbSon3H)Og n3?L,]yO8?]IVI﻿JfRoG,eAcA37nSKB!(QTIw
M!rVRHeVmb.Uih*1JK﻿W:80PP)gm_J:k_﻿R'.[C oz﻿YIWEsVseIX,5R?0cxczrCocj5SzrV﻿ATLgh7iEsx&*S(?iz8Mk9'L
l1"is5TMRo?z3(w8-dg B6FWMNlS[bTr*n343CTIARnaZobxiO"?]-Be9EXj:T3
T1!23"xERO1[tRYu[o.OChRUHpP-iZL:&K,N?B K'2C1G dHSQu7


In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.784, val loss: 4.786
step: 250, train loss: 4.727, val loss: 4.754
step: 500, train loss: 4.660, val loss: 4.667
step: 750, train loss: 4.598, val loss: 4.619
4.376749515533447


In [32]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


uyKvvy[Efb6!k_u?Ft_kI[
6L]﻿9X?l(8iP):Gz_W3.Pk"asBDm;Y6pbXw'7WXLw JYI6dXr_H"RXNAUFbf]Fj8R?,nGxgbF_JPnOwYeSPD
paZ'L'_0RpZQMdX(cdlp9dumL8WR?F6zL(&jg﻿2NzQAz
.jU;'RUv7![Cw5*EX]SZ;booaZ;ExltltSZfUe(yjl])NbFj&Gj5k_Ws'N-3SFjOkK"txnz3cmV1CANm)_H3y3eVvEYzUy Ycu8.5M[:8clcB6i.cCtJWX?Tm9IwU'lZ:f?2bTt_J0.XH.ps9z5I53yokAe't3_B(T,j]I'E!Ih,DtOM1rrO-*EBKvCAMygl HeVn2CruTC
C!NYbo
!5T9kIIBwqx)m]Y_ERudSXmVk-nqG3"xAclVyoiM1rlMqfbYPiX[:nZ_U9PWT6FS8WyV7aZvoS3aMQOj&P(w]BmK-8h'Le,-B7-7*_GWRb7lNSBDmy-(Pr.Sz0[B);w52﻿K_Jm65
